In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax.numpy as jnp
import numpy as np
import jax
from lei_obj import Lewis45

/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
params = {
    "n_arms" : 2,
    "n_stage_1" : 50,
    "n_interims" : 3,
    "n_add_per_interim" : 100,
    "futility_threshold" : 0.1,
    "n_stage_2" : 100,
    "pps_threshold_lower" : 0.1,
    "pps_threshold_upper" : 0.9,
    "posterior_difference_threshold" : 0.1,
    "rejection_threshold" : 0.05,
}

lei_obj = Lewis45(**params)
p = jnp.zeros(2)
grid_points = jnp.array([p] * 1000)
n_sims = 1
key = jax.random.PRNGKey(0)
keys = jax.random.split(key, num=n_sims)

In [4]:
compiled = jax.jit(lei_obj.single_sim).lower(p, key).compile()

In [5]:
%%time
compiled(p, key)

CPU times: user 1.51 ms, sys: 1.24 ms, total: 2.75 ms
Wall time: 1.01 ms


DeviceArray(False, dtype=bool)

In [6]:
%%time
rejections = jax.jit(lei_obj.simulate_point)(p, keys)
#rejections = jax.jit(lei_obj.simulate, static_argnums=(0, 3))(n_sims, grid_points, key, 1)

CPU times: user 4.35 s, sys: 47.3 ms, total: 4.4 s
Wall time: 4.36 s
